In [1]:
import tensorflow as tf
# import tf.keras as keras

In [2]:
from tensorflow import keras

In [3]:
import dlib

In [4]:
import cv2

In [5]:
import mtcnn

In [6]:
from mtcnn.mtcnn import MTCNN

In [7]:
model = MTCNN()

In [8]:
imag = cv2.imread('./myImage.jpeg')

In [9]:
cv2.imshow('image',imag)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
faces = model.detect_faces(imag)
for face in faces:
    print(face)

{'box': [275, 385, 382, 464], 'confidence': 0.99915611743927, 'keypoints': {'left_eye': (391, 559), 'right_eye': (570, 562), 'nose': (477, 658), 'mouth_left': (398, 745), 'mouth_right': (548, 750)}}


In [11]:
x,y,width,height =faces[0]['box']

In [19]:
rect = cv2.rectangle(imag,(x,y),(x+width,y+height),(0,0,255))
landmarks=faces[0]['keypoints']
for i in landmarks:
    rect=cv2.circle(rect,landmarks[i],3,(0,0,255))
cv2.imshow("image",rect)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [17]:
landmarks=faces[0]['keypoints']
for i in landmarks:
    print(landmarks[i])

(391, 559)
(570, 562)
(477, 658)
(398, 745)
(548, 750)


In [35]:
vid=cv2.VideoCapture(0)
processFrame = 0
while(True):
    ret,frame = vid.read()
    print(frame.shape)
    if processFrame%2 == 0:
        faces = model.detect_faces(frame)
        for face in faces:
            box = face['box']
            frame = cv2.rectangle(frame,(box[0],box[1]),(box[0]+box[2],box[1]+box[3]),(0,0,255))
            landmarks = face['keypoints']
            for i in landmarks:
                frame=cv2.circle(frame,landmarks[i],3,(0,0,255))
        cv2.imshow('detection',frame)
    processFrame = processFrame + 1
    if cv2.waitKey(1) & 0xFF ==ord('q'):
        break

vid.release()
cv2.destroyAllWindows()

(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
